In [1]:
import re

def extract_dram_utilized_capacity(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    in_dram = False
    current_section = None
    capacities = {
        'input': 0,
        'output': 0,
        'weight': 0
    }
    
    for line in lines:
        line = line.strip()

        # Detect the start of DRAM level
        if line.startswith("=== DRAM ==="):
            in_dram = True
            continue

        # Exit DRAM parsing if another level starts
        if in_dram and line.startswith("===") and not line.startswith("=== DRAM ==="):
            break

        # Identify current section
        if line.startswith("Inputs:"):
            current_section = 'input'
        elif line.startswith("Outputs:"):
            current_section = 'output'
        elif line.startswith("Weights:"):
            current_section = 'weight'

        # Extract Utilized capacity
        if in_dram and "Utilized capacity" in line:
            match = re.search(r'Utilized capacity\s+:\s+([0-9.]+)', line)
            if match and current_section:
                value = float(match.group(1))
                capacities[current_section] = value
    return capacities

In [2]:
def extract_total_chip_area(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    total_area = 0.0
    for line in lines:
        line = line.strip()
        if line.startswith("Area") and "um^2" in line:
            match = re.search(r'Area\s+\(total\)?\s*:\s*([0-9.]+)', line)
            if not match:
                match = re.search(r'Area\s*:\s*([0-9.]+)', line)
            if match:
                total_area += float(match.group(1))
    return total_area

In [3]:
def extract_total_energy(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip()
        if line.startswith("Energy:") and "uJ" in line:
            match = re.search(r'Energy:\s+([0-9.]+)\s*uJ', line)
            if match:
                return float(match.group(1))
    return None

In [6]:
def save_to_yaml(output_path, dram_caps, area, energy):
    with open(output_path, 'w') as f:
        f.write("dram_utilized_capacity:\n")
        f.write(f"  input: {dram_caps['input']}\n")
        f.write(f"  output: {dram_caps['output']}\n")
        f.write(f"  weight: {dram_caps['weight']}\n")
        f.write(f"total_chip_area_um2: {area}\n")
        f.write(f"total_energy_uJ: {energy}\n")

if __name__ == '__main__':
    file_path = 'timeloop-mapper.stats.txt'
    output_yaml = 'timeloop_summary.yaml'
    dram_caps = extract_dram_utilized_capacity(file_path)
    total_chip_area = extract_total_chip_area(file_path)
    total_energy = extract_total_energy(file_path)
    
    print("DRAM Utilized Capacities:")
    print(f"  Input Capacity  : {dram_caps['input']}")
    print(f"  Output Capacity : {dram_caps['output']}")
    print(f"  Weight Capacity : {dram_caps['weight']}")
    print(f"\nTotal Chip Area   : {total_chip_area:.2f} um^2")
    print(f"Total Energy      : {total_energy:.2f} uJ")

    save_to_yaml(output_yaml, dram_caps, total_chip_area, total_energy)
    print(f"\nYAML saved to: {output_yaml}")

DRAM Utilized Capacities:
  Input Capacity  : 151875.0
  Output Capacity : 401408.0
  Weight Capacity : 864.0

Total Chip Area   : 255599.30 um^2
Total Energy      : 59.28 uJ

YAML saved to: timeloop_summary.yaml
